In [2]:
class Config():
    '''
    - Playlists have been named Liked and Disliked, if you want to change names, change positive_class_playlist_name variabe to playlist name of liked songs
    - Change output path to where you want to store track features csv 
    - To get client_id and client_secret follow these steps:
        1. Goto https://developer.spotify.com/dashboard/
        2. Login using spotify credentials
        3. Select Create an App > Enter App name and description > Check the boxes > Create
        4. Open App > Copy Client ID and Client Secret below
        5. On the App page, select Edit Setting
        6. In the Redirect URL box, add http://localhost:8080 and save
    '''
    positive_class_playlist_name = 'Liked'  # change this to name of playlist with liked songs
    negative_class_playlist_name = 'my old tracks.' # change this to name of playlist with disliked songs
    output_path = r'/Users/syedrahimali/Desktop/spotify'    # change this to output path
    client_id = '***************'  # change this to your spotify account client id
    client_secret = '*********'  # change this to your spotify account client secret
    redirect_uri = 'http://localhost:8080'  # Don't change, redirects to localh

In [3]:
try:
    import spotipy
except:
    print('Install Spotipy by running command: `pip install spotipy --upgrade` on terminal')
from spotipy.oauth2 import SpotifyOAuth
import os
import csv
import sys
# from Config import Config

'''
- Make changes in config.py file before running this code
- Generates track_features.csv file
- You will need to login and authorize you developer app to access your spotify data while running the app for first time
- If you get bad token error on running, try deleting .cache(hidden) file in code's directory
'''

os.environ['SPOTIPY_CLIENT_ID'] = Config.client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = Config.client_secret
os.environ['SPOTIPY_REDIRECT_URI'] = Config.redirect_uri

class Track(object):
    '''
    - Generates Track instance
    - A track has id, name, dict features and dict analysis
    '''
    def __init__(self, track_id, track_name):
        self.id = track_id
        self.name = track_name
        self.features = sp.audio_features(self.id)[0]
        # self.analysis = sp.audio_analysis(self.id) # Take time to fetch, breaks with default request timeout of spotipy (=5)


class Playlist(object):
    '''
    - Generate Playlist instance
    - A playlist has id, name and a list of Tracks
    - Tracks are instances for Track class
    '''
    def fetch_tracks(self):
        for offset in range(0,800,100):
            for item in sp.playlist_items(self.id, offset=offset)['items']:
                self.tracks.append(Track(item['track']['id'],item['track']['name']))

    def get_track(self):
        return self.tracks

    def get_track_names(self):
            return [track.name for track in self.tracks]

    def get_selected_features(self,track):
        '''
        Returns a list of selected features of track along with playlist name and track class(1/0) i.e. 1 for Liked and 0 for Disliked
        '''
        return [track.id,track.name,track.features['duration_ms'],track.features['key'],track.features['mode'],track.features['time_signature'],track.features['acousticness'],track.features['danceability'],track.features['energy'],track.features['instrumentalness'],track.features['liveness'],track.features['loudness'],track.features['speechiness'],track.features['valence'],track.features['tempo'],self.name,(self.name == Config.positive_class_playlist_name) * 1]

    def get_track_features(self):
        '''
        Returns a list of lists of selected features of all the tracks in playlist
        '''
        return [self.get_selected_features(track) for track in self.tracks]

    def get_track_analysis(self):
        return [track.analysis for track in self.tracks]

    def __init__(self, playlist_id, playlist_name):
        self.id = playlist_id
        self.name = playlist_name
        self.tracks = []
        self.fetch_tracks()



scope = "playlist-read-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
playlists_json = sp.current_user_playlists(limit=50, offset=0)

playlists = []
for item in playlists_json['items']:
    playlists.append(Playlist(item['id'],item['name']))

track_features_list = [['ID','NAME','DURATION_MS','KEY','MODE','TIME_SIGNATURE','ACOUSTICNESS','DANCEABILITY','ENERGY','INSTRUMENTALNESS','LIVENESS','LOUDNESS','SPEECHINESS','VALENCE','TEMPO','PLAYLIST','CLASS']]
for playlist in playlists:
    if playlist.name == Config.positive_class_playlist_name or playlist.name == Config.negative_class_playlist_name:
        for playlist_track_features_list in playlist.get_track_features():
            track_features_list.append(playlist_track_features_list)


with open(os.path.join(Config.output_path,'track_feature.csv'),'w',newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_NONNUMERIC)
    wr.writerows(track_features_list)
